All essential modules from `labext` can be imported from `labext.prelude`.

In [1]:
import pandas as pd, numpy as np, ipywidgets as widgets, json

from IPython.display import HTML, Javascript
from labext.prelude import M, A, W

%load_ext autoreload
%autoreload 2

## Javascript modules

The Javascript (JS) modules are accessed via the `M` namespace in `labext.prelude`. A module needs to be registered before using. As an example, we register the `DataTable` module to have a better display for pandas DataFrame.

In [2]:
M.DataTable.register()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The `register` function will return a javascript code that modify your current HTML to inject required JS/CSS files (adding `script` and `link` tags). This function is safe to call repeatedly as it won't pollute your DOM.

You can also register multiple modules in one call using `M.register()` function

In [3]:
M.register([M.DataTable, M.Tippy])

<IPython.core.display.Javascript object>

### DataTable

This JS module provides a better display for the pandas DataFrame. To display the DataFrame beautifully, just call the render function of the module. Under the hood, the render function will construct a DataTable widget and render it.

This is different with previous `labext` version in which the DataFrame is converted automatically. By creating a widget for the DataFrame, we are able to handle large tables as we do not render the whole table, but only part of the table depends on the pagination parameters.

In [21]:
df = pd.DataFrame([
    ['<a href="http://google.com?q=john" style="color: blue" target="_blank">john</a>', '123-231-2321', 30],
    ['peter', '231-231-2321', 35],
], columns=['name', 'phone', 'age'])

M.DataTable.render(df)

HTML(value='', _dom_classes=('DataTable_b4d4f1ad2ece4c95a13140c4f0855321',))

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='b13b9c1e-17fc-4965-83b9-50d669a3481a')

<IPython.core.display.Javascript object>

In [24]:
df = pd.DataFrame([
    [f'row_{j}_{i}' for i in range(30)]
    for j in range(100)
], columns=[f'col_{i}' for i in range(30)])

M.DataTable.render(df, scrollX=True, lengthMenu=[[3, 5], [3, 5]])

HTML(value='', _dom_classes=('DataTable_b55c011d0f9d479aa626e18295fbba93',))

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='25c715cb-1f8f-410c-998c-52b4bb8fc0a4')

<IPython.core.display.Javascript object>

### Tippy

This JS module allows you to add additional content that will show only when hovering over a DOM element. The additional content is added via the HTML attribute `data-tippy-content`, if it contains HTML, you can unescape it by setting `data-tippy-allowHTML` to be `true`.

Similar to the `DataTable` module. This JS module does not render the content automatically, you have to call `M.Tippy.render()` function after rendering the HTML as in the following example.

In [6]:
M.Tippy.register()

<IPython.core.display.Javascript object>

In [43]:
display(HTML('<button data-tippy-content="<b>Hello</b>, what are you doing?" data-tippy-allowHTML="true">Button</button>'))
M.Tippy.render()

<IPython.core.display.Javascript object>

Similar to the **DataTable** module, we can also configure the module for our need. For example, we may want the pop-over to not disappear so that we can interact with the content such as clicking the link.

All configurations are available on [**Tipply**'s website](https://atomiks.github.io/tippyjs/v6/all-props/).

In [40]:
M.Tippy.set_args(interactive=True, placement="right")

In [44]:
# we need the set the height of the container, as interactive Tippy's content is behind it, not like non-interactive content
popover = "<b>Hello</b>, please click to visit <a href='https://google.com' target='_blank'>google.com</a>"
display(HTML(f'<div style="height: 25px"><button data-tippy-content="{popover}" data-tippy-allowHTML="true">Button</button></div>'))
M.Tippy.render()

<IPython.core.display.Javascript object>

## Pre-built UI components

This library provides some UI components allowing us to interactive annotate/explore data. These components can be accessed via the `A` namespace.

### Slider

This component is for displaying an item in a list, and changing the item interactively. You need to provide a function that render the item given its index. The range of the slider is defined by two parameters: `min` (default 0) and `max` (inclusive).

In [5]:
lst = ["google.com", "amazon.com", "microsoft.com"]

def render(index):
    item = lst[index]
    display(HTML(f"<a href='https://{item}' target='_blank'>{item}</a>"))
    
A.slider(render, max=len(lst) - 1)

Output()

### Annotator

This component is for annotating a list of examples. Each example instance needs to have an `id` property (type `str`), and a `render` function to display the example. You can create a class that inherits the `A.annotators.Example` class to make sure that you implement the correct methods, and for python 3 type check to not complain, but it's completely optional.

Below is a simple example of classifying positive and negative reviews.

In [6]:
class MyExample:
    def __init__(self, id, review):
        self.review = review
        self.id = str(id)
        
    def render(self):
        display(HTML("<p>" + self.review + "</p>"))
        
reviews = ["It broke after two days", "It works as expected", "Good price"]
examples = [MyExample(i, review) for i, review in enumerate(reviews)]

This library provides basic annotators for the classification task: `A.annotators.ClassificationAnnotator` and `A.annotators.BatchClassificationAnnotator` (display multiple examples in one page). The annotated data is stored in a given file so you do not need to worry about losing your progress. The output file is in the CSV format, where the first column is the example id, and the second column is the label that you assigned.

If you need to customize these annotators (for example, assign multiple classes to one example), you can implement the abstract class `A.annotators.Annotator` or `A.annotators.PersistentAnnotator`. For more details, you can read the documentation in these classes.

In [7]:
outfile = "/tmp/labeled_data.csv"
annotator = A.annotators.ClassificationAnnotator(outfile, examples, ["positive", "negative"])
annotator.render()

Output(layout=Layout(border='1px solid #d24829', margin='0 0 0 -2px', padding='2px'))

In [8]:
outfile = "/tmp/labeled_data.csv"
annotator = A.annotators.BatchClassificationAnnotator(outfile, examples, ["positive", "negative"], batch_size=3)
annotator.render()

Output(layout=Layout(border='1px solid #d24829', margin='0 0 0 -2px', padding='2px'))

You can get the annotated result directly from the file, or from the `annotator.labeled_examples` property

In [9]:
annotator.labeled_examples

{'0': 'negative', '1': 'negative', '2': 'negative'}

In [10]:
!cat {outfile}

0	negative
1	negative
2	negative


## Additional widgets

This library also provides some primitive widgets that you can use to build your own components. Those widgets can be imported via the `W` namespace.

A widget may require some JS modules, so you need to registered them as in the following examples:

In [11]:
M.register(W.HideableButton.required_modules())

<IPython.core.display.Javascript object>

### HideableButton

In [12]:
import ipywidgets as widgets

def render():    
    display(widgets.Button(description='another button'))
    
display(W.HideableButton('click to show', render, show_event='click').widget)

### Selection

In [13]:
M.register(W.Selection.required_modules())

<IPython.core.display.Javascript object>

In [14]:
def search_company(query, _cache={}):
    if 'data' not in _cache:
        _cache['data'] = [{"text": text, "value": str(i)} for i, text in enumerate(['google', 'amazon', 'facebook', 'microsoft', 'tesla', 'uber']) ]
    # assuming that we did a search
    return _cache['data']
    
w = W.Selection(search_fn=search_company, layout={'width': '250px'}, max_items=2, default_value=['1', '0'])
display(w.widget)
display(*w.get_auxiliary_components())

HTML(value='<select></select>', layout=Layout(width='250px'), _dom_classes=('selection_ae361cdf1f6c448293dd716…

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''))

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''))

<IPython.core.display.Javascript object>

To set/get the value of this widget on the python side, use function `get_value`, `set_value`. To clear the value of the widget, set its value to empty string. You can also listen to the `on_change` event by setting your own callback: `w.on_change(callback)`.

In [15]:
w.get_value()

['1', '0']

In [16]:
w.set_value(["0", "2"])